<a href="https://colab.research.google.com/github/alexacheui/ateema_capstone/blob/main/Deterministic_classed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import os
os.chdir('/content/drive/My Drive/ADSP Capstone')

In [10]:
!pwd

/content/drive/.shortcut-targets-by-id/1wMbMX076_Mwy4gtzLGnBd94PedqtndDq/ADSP Capstone


In [11]:
!pip install moviepy opencv-python --quiet

In [12]:
from moviepy.editor import VideoFileClip, ImageClip, CompositeVideoClip, concatenate_videoclips
import cv2
import numpy as np
import requests
from PIL import Image
import proglog
from IPython.display import HTML
from base64 import b64encode


In [13]:
class VideoProcessor:
    def __init__(self, video_path, image_urls, output_path):
        self.video_path = video_path
        self.image_urls = image_urls
        self.output_path = output_path

    class SilentLogger(proglog.ProgressBarLogger):
        def bars_callback(self, bar, attr, value, old_value=None):
            pass

    def download_image(self, url, path):
        response = requests.get(url)
        with open(path, 'wb') as file:
            file.write(response.content)

    def get_extension(self, url):
        return url.split("?")[0].split(".")[-1].lower()

    def convert_to_rgb_png(self, image_path):
        img = Image.open(image_path).convert("RGB")
        png_path = os.path.splitext(image_path)[0] + ".png"
        img.save(png_path, "PNG")
        return png_path

    def create_mask(self, frame, lower_bound, upper_bound):
        hsv = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
        mask = cv2.inRange(hsv, lower_bound, upper_bound)
        mask_inv = cv2.bitwise_not(mask)
        return mask_inv

    def process_frame(self, frame):
        lower_bound = np.array([35, 100, 100])
        upper_bound = np.array([85, 255, 255])
        mask = self.create_mask(frame, lower_bound, upper_bound)
        mask_3ch = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
        fg = cv2.bitwise_and(frame, mask_3ch)
        return fg

    def prepare_backgrounds(self):
        image_paths = []
        for i, url in enumerate(self.image_urls):
            extension = self.get_extension(url)
            path = f"background{i + 1}.{extension}"
            self.download_image(url, path)
            png_path = self.convert_to_rgb_png(path)
            image_paths.append(png_path)
        return image_paths

    def create_video(self):
        image_paths = self.prepare_backgrounds()

        video = VideoFileClip(self.video_path).subclip(0, 30)
        processed_video = video.fl_image(self.process_frame)

        video_width, video_height = 640, 360

        background_clips = [ImageClip(path).resize((video_width, video_height)).set_duration(10) for path in image_paths]
        background = concatenate_videoclips(background_clips)

        processed_video = processed_video.resize(width=200).set_pos((15, "center"))

        final_video = CompositeVideoClip([background, processed_video])
        final_video.write_videofile(self.output_path, codec="libx264", logger=self.SilentLogger())

    def display_video(self):
        video_path = f'/content/drive/MyDrive/ADSP Capstone/{self.output_path}'
        mp4 = open(video_path, 'rb').read()
        data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
        return HTML(f'''
        <video width=600 controls>
            <source src="{data_url}" type="video/mp4">
        </video>
        ''')



In [14]:

# Usage example
image_urls = [
    "https://cdn.choosechicago.com/uploads/2021/06/43d322b9-1571-44d9-8c2b-0f04ff0caa7d-1-900x630.jpeg",
    "https://www.visitchicagonorthshore.com//images/blog/Apr24Blog1_SkokieScuplture.jpg",
    "https://cdn.choosechicago.com/uploads/2020/01/Blue-Door-Kitchen-patio_photo-by-Skylar-Taft-900x593.png"
]



In [15]:
video_processor = VideoProcessor("avatar.mp4", image_urls, "test.mp4")
video_processor.create_video()
video_processor.display_video()

1. Instantiate the VideoProcessor class with the required video path, image URLs, and output path
2. Call create_video() to generate the video
3. Display_video() to display it